In [1]:
import sys
import math
import csv
import random
import json 
import time

try:
    jparams = json.load(open('params.json'))
except:
    print("ERROR: something is wrong with the params.json file.")
    sys.exit()
    
#-- store the input 3D points in list
list_pts_3d = []
with open(jparams['input-file']) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    header = next(r)
    for line in r:
        p = list(map(float, line)) #-- convert each str to a float
        assert(len(p) == 3)
        list_pts_3d.append(p)

In [2]:
import math
import numpy
import scipy.spatial
import startin

In [3]:
class BoundingBox:
    """A 2D bounding box"""
    
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for x, y, *_ in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y
            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y
                
    @property
    def width(self):
        return self.maxx - self.minx
    @property
    def height(self):
        return self.maxy - self.miny
    def __repr__(self):
        return "BoundingBox(X: {} - {}, Y: {} - {})".format(
            self.minx, self.maxx, self.miny, self.maxy)
      
bbox = BoundingBox(list_pts_3d)
bbox

BoundingBox(X: 2.0 - 252.0, Y: 3.0 - 253.0)

In [4]:
class Raster:
  """Simple raster based on ESRI ASCII schema"""
  
  def __init__(self, bbox, cell_size, no_data=-9999):
    self.ncols = math.ceil(bbox.width / cell_size)
    self.nrows = math.ceil(bbox.height / cell_size)
    self.xllcorner = bbox.minx
    self.yllcorner = bbox.miny
    self.cell_size = cell_size
    self.no_data = no_data
    
    # initialize list of values with no_data
    self.values = [self.no_data] * self.ncols * self.nrows
    
  @property
  def centers(self):
    """Cell center coordinates"""
    xulcenter = self.xllcorner + self.cell_size/2
    yulcenter = self.yllcorner - self.cell_size/2 + self.cell_size*self.nrows
    for i in range(self.nrows):
      for j in range(self.ncols):
        x = xulcenter + j * self.cell_size
        y = yulcenter - i * self.cell_size
        yield (x,y)
    
  def to_ascii(self):
    rows = [
      "NCOLS %d" % self.ncols,
      "NROWS %d" % self.nrows,
      "XLLCORNER %s" % self.xllcorner,
      "YLLCORNER %s" % self.yllcorner,
      "CELLSIZE %s" % self.cell_size,
      "NODATA_VALUE %s" % self.no_data,
      ' '.join( (str(x) for x in self.values) )
    ]
    return '\n'.join(rows)

In [5]:
bbox = BoundingBox(list_pts_3d)
raster = Raster(bbox, 30)
type(raster.to_ascii())

str

In [91]:
# gaussian function for kriging
_nugget = 0
_sill = 1400
_range = 300
gamma = lambda x: _nugget+_sill*(1.0 - math.exp(-9.0*x*x/(_range**2)))
dist = lambda a, b: math.sqrt( (a[0]-b[0])**2 + (a[1]-b[1])**2 )    

list_pts_2d = [(x,y) for x,y,z in list_pts_3d]
list_pts_z = [(z) for x,y,z in list_pts_3d]
kdtree = scipy.spatial.KDTree(list_pts_2d)
dt = startin.DT()
dt.insert(list_pts_2d)
                                                                                          
raster.values = []
for center in raster.centers:
    # catch outside on convex hull case
    if not dt.locate(*center):
        raster.values.append(raster.no_data)
        continue
    # get samples in radius
    samples = kdtree.query_ball_point(center, 10)
    coords = [list_pts_2d[i] for i in samples]
    values = [list_pts_z[i] for i in samples]
    dists = [math.sqrt( (center[0]-c[0])**2 + (center[1]-c[1])**2 ) 
            for c in coords]
    # catch no sample in radius case
    if not samples:
        raster.values.append(raster.no_data)
        continue
    # catch zero distance case
    if 0 in dists:
        i = dists.index(0)
        raster.values.append(values[i])
        continue 
    
    # create covarience matrix using gaussian dist function
    cov_mat = []
    for i in coords:
      row = []
      for j in coords:
        row.append(gamma(dist(i, j)))
      cov_mat.append(row+[1])
    cov_mat.append([1]*len(coords)+[0])
    cov_mat = numpy.array(cov_mat)
    # create d matrix gaussian dist function
    d_mat = []
    for i in coords:
        d_mat.append([gamma(dist(i, center))])
    d_mat.append([1])
    d_mat = numpy.array(d_mat)
    # calculate weight matrix
    try:
      w_mat = numpy.matmul(numpy.linalg.inv(cov_mat), d_mat)
    except:
      print('singular')
      # catch singular matrix case
      raster.values.append(raster.no_data)
      continue
    weights = [w[0] for w in w_mat[:-1]]
    weights_norm = [w/sum(weights) for w in weights]
    result = sum([val*w for val, w in zip(values, weights_norm)])
    raster.values.append(result)
        
raster.values

singular
singular
singular


[-9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 69.66416309036346,
 70.04631616264459,
 76.76730020554187,
 94.66804017832457,
 89.16785332004889,
 67.68182239910485,
 42.059744353024776,
 38.99493321679758,
 -9999,
 88.36722176839922,
 93.28997251724941,
 108.52649169701697,
 113.8682928423466,
 98.50434000496872,
 68.5678281681976,
 49.17729408433547,
 49.44423912928414,
 -9999,
 109.61338125397168,
 113.75,
 129.2871211892162,
 128.04349731995504,
 98.08100307439331,
 80.7022543112343,
 75.59510964173982,
 51.187059941839806,
 -9999,
 -9999,
 121.84711743179199,
 138.03194958000086,
 141.8037219368105,
 122.8909860320613,
 105.49336885754596,
 79.54935309667889,
 52.19812867655759,
 -9999,
 -9999,
 120.6367794573648,
 135.32779831766248,
 150.59602659234696,
 148.3851,
 117.87064502582855,
 90.03992317730075,
 65.55556,
 -9999,
 94.93072265693436,
 102.38594,
 123.18290613213806,
 137.17

In [59]:
test = ['a', 'b', 'c']
cov_mat = []
for i in test:
  row = []
  for j in test:
    row.append(i+j)
  cov_mat.append(row+[1])
cov_mat.append([1]*len(test)+[0])

cov_mat = numpy.array(cov_mat)
cov_mat

array([['aa', 'ab', 'ac', '1'],
       ['ba', 'bb', 'bc', '1'],
       ['ca', 'cb', 'cc', '1'],
       ['1', '1', '1', '0']], dtype='<U2')

In [60]:
point = 'z'
test = ['a', 'b', 'c']
d_mat = []
for i in test:
    d_mat.append([point+i])

d_mat = numpy.array(d_mat)
d_mat

array([['za'],
       ['zb'],
       ['zc']], dtype='<U2')

array([['aa', 'ab', 'ac'],
       ['ba', 'bb', 'bc'],
       ['ca', 'cb', 'cc']], dtype='<U2')

In [76]:
list(range(3))

[0, 1, 2]